# Integrantes:

* Andresa Bicudo
* Gabriel Yamashita
* Leonardo Malta

----

# Introdução 
A pandemia do corona vírus fez com que o futebol parasse, inclusive o campeonato inglês (Premiere League). Existem campeonatos que foram interrompidos, como o campeonato francês, mesmo sem ter finalizado todos os jogos e também há campeonatos que retonaram porém com desempenhos diferentes de cada equipe. Isso fez com que surgisse o seguinte questionamento: o que provavelmente ocorreria com o campeonato inglês se a pandemia não tivesse a interrompido?

----

# Objetivo:
Prever o restante do campeonato inglês com base no desempenho e estatísitcas das equipes até a 30ª rodada. 

----

# Imports:

In [1]:
# Importanto Bibliotecas para o Trabalho:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import statsmodels.api as sm

import scipy as scp
import sklearn

from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

----

# Separação dos Dados:

In [3]:
# carregando os dados do Excell para o pandas
dados = pd.read_excel('masterdata.xlsx')
dados.head(5)

FileNotFoundError: [Errno 2] No such file or directory: 'masterdata.xlsx'

In [ ]:
# separando os resultados em uma nova coluna e em números (0, 1 e 2)

def Determina_Resultado(df, home, away):
    if df[home] > df[away]: # se o time local ganhou do visitante
        return 0
    elif df[home] < df[away]: # se o time visitante ganhou do local
        return 2
    elif df[home] == df[away]: # se empatou
        return 1

# criando uma coluna com os números que representam os resultados
dados['Result'] = dados.apply(
    lambda dados: Determina_Resultado(dados, 'Score_home', 'Score_away'), axis=1)

# testando se deu certo
dados.head(5)

----

# Obtendo Variáveis de Interesse  

In [ ]:
# Definindo variáveis importantes: 
# removendo as colunas que não serão necessárias para calcular as probabilidades
stats = dados.drop(columns=[
    'MatchID', 'Home_team', 'Away_team', 'Score_home', 'Score_away',
    'year', 'Result'])

# definindo as colunas dos nomes dos times
games = dados.loc[: , ['Home_team', 'Away_team']]

# definindo as colunas do resultado (em 0, 1 e 2)
results = dados.loc[: , ['Result']]

stats.head(5)

In [ ]:
# Obtendo a correlação dos dados para times que jogam em seu estádio:
home_features = [i for i in list(stats) if '_home' in i]
corr = stats[home_features].corr()
sns.heatmap(corr, cmap="Blues")

# Fatores relevantes

A partir da análise do heatmap, é possível saber se duas variáveis estão medindo características similares. 
Por exemplo, a quantidade de passes ("touches_home") está muito interligada com a posse de bola. 
Para eliminar as variáveis que se relacionam, usa-se o **"Variance Inflation Factor"**, que é um fator que indentifica multicolinearidade de duas variáveis. 
**Valores maiores que 5** significam que há muita correlação. 

Mais informações em: https://www.statisticshowto.com/variance-inflation-factor/.

In [ ]:
# Enumera elementos de uma lista para percorrer com o for
enumerate(list(stats))

In [ ]:
# detectando a multicolinearidade entre características a partir do fator de inflação de variância
# armazenando esses valores numa lista
variance_inflation = []
for i, feature in enumerate(list(stats)):
    vif_tup = (feature, variance_inflation_factor(stats.values, i))
    variance_inflation.append(vif_tup)

    
# sabendo que os fatores maiores de 5 possuem alta multicolineariade
# selecionaremos apenas os que são menores (ou iguais, mas nesse caso não há diferença) que 5
# armazenando os relevantes em uma lista
relevant_features_vi = []
for i in range(len(variance_inflation)):
    if variance_inflation[i][1] <= 5:
        relevant_features_vi.append(variance_inflation[i][0])


print('Número de itens relevantes: {0}\n'.format(len(relevant_features_vi)))
print(relevant_features_vi)

# Random Forest - Tentativa 1 

O Random Forest é um método de Regressão baseado em várias árvores de probabilidade. Cada uma delas faz as probabilidades trocando a ordem dos fatores. Caso apresente algum erro, ele será minimizado.

In [ ]:
# Teste sem retirada das variáveis
X = stats.values
y = results.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=42)

classificador_RF = RandomForestClassifier(n_estimators=500, random_state=42)
classificador_RF.fit(X_train, y_train)

In [ ]:
RF_y_pred = classificador_RF.predict(X_test)
print('Acurácia do modelo Random Forest com todas as variáveis:',
      accuracy_score(y_test, RF_y_pred))

# Random Forest - Tentativa 2 

In [ ]:
# Predição com variáveis com variance inflation menores que 5:
RF2_X = (stats.loc[:, relevant_features_vi]).values
RF2_y = results.values

In [ ]:
RF2_X_train, RF2_X_test, RF2_y_train, RF2_y_test = train_test_split(
    RF2_X, RF2_y, test_size=.25, random_state=42)

classificador_RF2 = RandomForestClassifier(n_estimators=500, random_state=42)
classificador_RF2.fit(RF2_X_train, RF2_y_train)

In [ ]:
RF2_y_pred = classificador_RF2.predict(RF2_X_test)
print(
    'Acurácia do modelo Random Forest somente com as variáveis determinadas pelo variance inflation :',
    accuracy_score(RF2_y_test, RF2_y_pred))

A conclusão que pode ser feita é a de que a "variance inflation" não é um bom parâmetro para determinar variáveis importantes. 

### Nova Seleção de Melhores Variáveis

In [ ]:
forest = RandomForestClassifier(n_estimators=500,random_state=42)
forest.fit(X_train, y_train)

features = stats.columns
importances = forest.feature_importances_

indices = np.argsort(importances)[::-1]
indices
for f in range(X_train.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30,features[indices[f]], importances[indices[f]]))

In [ ]:
# Prevendo o melhor resultado por meio do Random Forest 
def Resultado_RF(lista_variaveis, quantidade):
    relevant_features = []
    for i in range(len(lista_variaveis)): #features[indices]
        if i <= quantidade:
            relevant_features.append(lista_variaveis[i])
        else:
            pass
    
    RF3_X = (stats.loc[:, relevant_features]).values
    RF3_y = results.values

    RF3_X_train, RF3_X_test, RF3_y_train, RF3_y_test = train_test_split(
        RF3_X, RF3_y, test_size=.25, random_state=42)

    classificador_RF = RandomForestClassifier(n_estimators=500, random_state=42)
    classificador_RF.fit(RF3_X_train, RF3_y_train)

    RF3_y_pred = classificador_RF.predict(RF3_X_test)
    
    return accuracy_score(RF3_y_test, RF3_y_pred)

# Teste Função
Resultado_RF(features[indices], 1)

In [ ]:
scores = []
for i in range(len(features[indices])):
    scores.append(Melhor_Resultado_RF(features[indices], i))
    
scores

In [ ]:
index(max(scores))

----

# Regressão Multinomianal - Tentativa 1

In [ ]:
logit_model = sm.MNLogit(endog=y_train.flatten(), exog=X_train)
result = logit_model.fit()

stats1 = result.summary()
stats2 = result.summary2()

print(stats1)

In [ ]:
print(stats2)

In [ ]:
def Determina_Acuracia_Multinominal(predict, teste_y):
    lista_maiores = [max(i) for i in predict]
    lista_index = []
    for i in range(len(predict)):
        for p in range(3):
            if predict[i][p] == lista_maiores[i]:
                lista_index.append(p)

    flat_list_of_results = []
    for sublist in np.ndarray.tolist(teste_y):  #y_test
        for item in sublist:
            flat_list_of_results.append(item)

    comparativo_rm = pd.DataFrame({
        'Resultado': flat_list_of_results,
        'Modelo': lista_index
    })

    corretos = 0
    for i in range(len(comparativo_rm['Resultado'])):
        if comparativo_rm['Resultado'][i] == comparativo_rm['Modelo'][i]:
            corretos += 1

    accuracy = corretos / (len(comparativo_rm['Resultado']))

    print(
        'A acurácia desse modelo de Regressão Multinominal é: {}'
        .format(accuracy))

In [ ]:
RM_pred = result.predict(exog=X_test)
Determina_Acuracia_Multinominal(RM_pred, y_test)

# Regressão Multinominal - Tentativa 2

Por meio dos valores de "p", conclui-se que  serão eliminadas as variáveis que são maiores que 0.1 em pelo menos duas das listas do summary, ou seja: (x6, x7 x8, x11, x12, x15, x18, x20, x21, x22, x26, x27, x28, x29, x30, x31).

In [ ]:
RM2_index_inrrelevant = [5, 6, 7, 10, 11, 14, 17, 19, 20, 21, 25, 26, 27, 28, 29, 30, 31]
RM2_relevant_features = list(
    stats.drop(stats.columns[RM2_index_inrrelevant], axis=1).columns)

RM2_X = (stats.loc[:, RM2_relevant_features]).values
RM2_y = results.values

RM2_X_train, RM2_X_test, RM2_y_train, RM2_y_test = train_test_split(
    RM2_X, RM2_y, test_size=.25, random_state=42)

RM2_logit_model = sm.MNLogit(endog=RM2_y_train.flatten(), exog=RM2_X_train)
RM2_result = RM2_logit_model.fit()

RM2_stats1 = RM2_result.summary()
RM2_stats2 = RM2_result.summary2()

print(RM2_stats1)

In [ ]:
print(RM2_stats2)

In [ ]:
# Predição 2 - Regressão Multinominal (Retirada de algumas variáveis)
RM2_pred = RM2_result.predict(exog=RM2_X_test)
Determina_Acuracia_Multinominal(RM2_pred, RM2_y_test)

O modelo obteve menos acertos ao passo que as variáveis com valor p maior que 0.1 foram retiradas.

----

# Regressão Naive Bayes Multinomial - Tentativa 1

In [ ]:
classificador_NB_multinomial = MultinomialNB()
classificador_NB_multinomial.fit(X_train, y_train)

In [ ]:
NB_y_pred = classificador_NB_multinomial.predict(X_test)
print('Acurácia do modelo Naive Bayes Multinomial:', accuracy_score(y_test, NB_y_pred))

# Regressão Naive Bayes Multinomial - Tentativa 2

In [ ]:
X1 = (stats.loc[: , relevant_features]).values
y1 = results.values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size = .25, random_state = 42)

In [ ]:
classificador_NB_multinomial = MultinomialNB()
classificador_NB_multinomial.fit(X_train, y_train)

In [ ]:
y_pred = classificador_NB_multinomial.predict(X_test)
print('Acurácia do modelo Naive Bayes Multinomial:', accuracy_score(y_test, y_pred))

----

# KNeighbors Classifier (Tentativa 1):

In [ ]:
X = stats.values
y = results.values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .25, random_state = 42)

In [ ]:
k_range = range(1,1001)
max_k = 0
max_k_value = 0
acuracia = []


for i in range(len(k_range)):
    classificador_KNeigh = KNeighborsClassifier(n_neighbors=i)
    classificador_KNeigh.fit(X_train, y_train)
    y_pred = classificador_KNeigh.predict(X_test)
    acuracia.append(metrics.accuracy_score(y_test, y_pred))
    
for i in range(k_range):
    if max_k < acuracia[i]:
        max_k_value = i
    
plt.plot(k_range, scores)
plt.xlabel('Value of K for KNN')
plt.ylabel('Testing Accuracy')

In [ ]:
y_pred = classificador_KNeigh.predict(X_test)
print('Acurácia do modelo K Neighbors:', accuracy_score(y_test, y_pred))

# KNeighbors Classifier (Tentativa 2):

In [ ]:
X1 = (stats.loc[: , relevant_features]).values
y1 = results.values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size = .25, random_state = 42)

In [ ]:
classificador_KNeigh = KNeighborsClassifier(n_neighbors=25)
classificador_KNeigh.fit(X_train, y_train)

In [ ]:
y_pred = classificador_KNeigh.predict(X_test)
print('Acurácia do modelo K Neighbors:', accuracy_score(y_test, y_pred))

----

# Gradient Boosting Classificador (Tentativa 1):

In [ ]:
X = stats.values
y = results.values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .25, random_state=0)

In [ ]:
classificador_Gradiente = GradientBoostingClassifier(random_state=0)
classificador_Gradiente.fit(X_train, y_train)

In [ ]:
y_pred = classificador_Gradiente.predict(X_test)
print('Acurácia do modelo Gradient Boosting:', accuracy_score(y_test, y_pred))

# Gradient Boosting Classificador (Tentativa 2):

In [ ]:
X1 = (stats.loc[: , relevant_features]).values
y1 = results.values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size = .25, random_state=0)

In [ ]:
classificador_Gradiente = GradientBoostingClassifier(random_state=0)
classificador_Gradiente.fit(X_train, y_train)

In [ ]:
y_pred = classificador_Gradiente.predict(X_test)
print('Acurácia do modelo Gradient Boosting:', accuracy_score(y_test, y_pred))

----

# Fazendo Predições com o Modelo

In [ ]:
def Historico_Confrontos(df, home, away, result_model):
    matches = df.loc[(df['Home_team'] == home) & (df['Away_team'] == away), :]
    resumo = matches.loc[:, relevant_features].mean().to_frame().T
    resultado = result_model.predict(exog=resumo)
    return resultado


Historico_Confrontos(dados, 'Everton', 'Liverpool', result)